In [ ]:
import os, time, hashlib
import requests

# 테스트용 더미 PDF 파일 생성
test_file = "/tmp/mock_drm_path/test.pdf"
os.makedirs(os.path.dirname(test_file), exist_ok=True)
with open(test_file, "wb") as f:
    f.write(b"Dummy PDF content for DRM removal test.")

# API 요청 설정
url = "http://rpa-wlm-service-basic-prd.api.hcp01.skyhinix.com/rpa-wlm-service/v1/api/automations/schedule/chatbot"
headers = {"X-enviroment": "PRD"}
payload = {
    "CrUri": "/tmp/mock_drm_path/test.pdf",      # DRM 제거 대상 파일의 경로 또는 URI
    "AutomationName": "DRMRemovalAutomation",    # 자동화 이름 (예시)
    "FileName": "test.pdf",                      # 파일명
    "ScheduleType": "IMMEDIATE",                # 즉시 실행 스케줄 (예시값)
    "BotType": "RPA",                            # 봇 유형 (예시값)
    "PoolName": "DefaultPool",                   # 실행할 RPA 봇 풀 이름 (예시)
    "UserInput": {}                              # 추가 입력값 없으면 빈 객체
}

# 호출 전 파일 상태 기록 (시간 및 해시)
mtime_before = os.path.getmtime(test_file)
hash_before = hashlib.md5(open(test_file, "rb").read()).hexdigest()
print(f"Before API call: mtime={mtime_before}, MD5={hash_before}")

# DRM 제거 API 호출
response = requests.post(url, headers=headers, json=payload)
print(f"API Response Code: {response.status_code}")
if response.status_code != 200:
    print("API 요청 실패 또는 오류:", response.text)

# 호출 후 파일 상태 기록 (시간 및 해시)
mtime_after = os.path.getmtime(test_file)
hash_after = hashlib.md5(open(test_file, "rb").read()).hexdigest()
print(f"After API call: mtime={mtime_after}, MD5={hash_after}")

# 파일 수정 여부 판단
if mtime_after != mtime_before or hash_before != hash_after:
    print("파일이 DRM 제거 API에 의해 수정(덮어쓰기)되었습니다.")
else:
    print("파일에 변화가 없었습니다. DRM 제거 실패 또는 파일 미변경.")


In [ ]:
import fitz  # PyMuPDF

def extract_images_from_pdf(pdf_path, output_dir):
    """PDF의 각 페이지를 이미지로 렌더링하여 저장하고, 파일 경로 리스트를 반환"""
    doc = fitz.open(pdf_path)
    image_paths = []
    for page_index in range(len(doc)):
        page = doc.load_page(page_index)
        # 페이지를 Pixmap (래스터 이미지)으로 변환 (dpi나 matrix로 해상도 조절 가능)
        pix = page.get_pixmap()  
        img_path = os.path.join(output_dir, f"page_{page_index+1}.png")
        pix.save(img_path)
        image_paths.append(img_path)
    doc.close()
    return image_paths


In [ ]:
import zipfile

def extract_images_from_docx(docx_path, output_dir):
    """DOCX 파일에서 임베디드 이미지를 추출하여 저장하고 경로 리스트 반환"""
    image_paths = []
    # DOCX는 ZIP 포맷이므로 ZipFile로 열기
    with zipfile.ZipFile(docx_path, 'r') as z:
        # 모든 파일 중 word/media 폴더 아래 파일 탐색
        for file_name in z.namelist():
            if file_name.startswith("word/media/"):
                # 원본 파일명을 이용하여 출력 경로 결정
                _, image_filename = file_name.split("word/media/")
                output_path = os.path.join(output_dir, image_filename)
                z.extract(file_name, output_dir)
                # Zip에서 추출하면 output_dir/word/media/ 경로에 생기므로, 이를 output_dir로 이동 또는 경로 재설정
                extracted_path = os.path.join(output_dir, file_name)
                os.replace(extracted_path, output_path)  # media 폴더 -> output_dir 직책
                image_paths.append(output_path)
    return image_paths


In [ ]:
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE

def extract_images_from_pptx(pptx_path, output_dir):
    """PPTX 파일의 모든 슬라이드에서 이미지를 추출하여 저장하고 경로 리스트 반환"""
    prs = Presentation(pptx_path)
    image_paths = []
    slide_index = 0
    for slide in prs.slides:
        slide_index += 1
        image_count = 0
        for shape in slide.shapes:
            # Placeholder나 텍스트 상자가 아닌 그림만 대상
            if shape.shape_type == MSO_SHAPE_TYPE.PICTURE:
                image_count += 1
                # 이미지 바이너리와 확장자 얻기
                image_data = shape.image.blob
                image_ext = shape.image.ext  # 확장자, 예: 'png', 'jpeg'
                output_path = os.path.join(output_dir, f"slide{slide_index}_image{image_count}.{image_ext}")
                with open(output_path, 'wb') as img_file:
                    img_file.write(image_data)
                image_paths.append(output_path)
    return image_paths


In [ ]:
def extract_images_from_xlsx(xlsx_path, output_dir):
    """XLSX 파일에서 임베디드 이미지를 추출하여 저장하고 경로 리스트 반환"""
    image_paths = []
    with zipfile.ZipFile(xlsx_path, 'r') as z:
        for file_name in z.namelist():
            if file_name.startswith("xl/media/"):
                _, image_filename = file_name.split("xl/media/")
                output_path = os.path.join(output_dir, image_filename)
                z.extract(file_name, output_dir)
                # 추출된 경로를 output_dir로 정리
                extracted_path = os.path.join(output_dir, file_name)
                os.replace(extracted_path, output_path)
                image_paths.append(output_path)
    return image_paths


In [ ]:
import requests
import mimetypes

def preprocess_document_file(input_path_or_url, output_dir, remove_drm=False):
    """
    주어진 문서 파일 경로 또는 URL에 대해:
    1) URL일 경우 파일을 다운로드 받고,
    2) 필요시 DRM 제거 API 호출을 수행한 후,
    3) 파일 유형별로 이미지 추출을 진행.
    최종적으로 추출된 이미지 경로들의 리스트를 반환.
    """
    # 1. 파일 다운로드 (URL인 경우)
    if input_path_or_url.startswith("http://") or input_path_or_url.startswith("https://"):
        url = input_path_or_url
        local_path = os.path.join(output_dir, os.path.basename(url.split('?')[0]))
        resp = requests.get(url)
        resp.raise_for_status()
        with open(local_path, 'wb') as f:
            f.write(resp.content)
    else:
        # 이미 로컬 경로인 경우
        local_path = input_path_or_url

    # 2. 파일 형식 확인 (확장자 기반)
    file_ext = os.path.splitext(local_path)[1].lower()
    # 만약 확장자로 확인이 어려우면 mimetypes 사용
    if file_ext == "":
        mime_type, _ = mimetypes.guess_type(local_path)
        if mime_type:
            if "pdf" in mime_type:
                file_ext = ".pdf"
            elif "presentation" in mime_type or "ppt" in mime_type:
                file_ext = ".pptx"
            elif "word" in mime_type or "doc" in mime_type:
                file_ext = ".docx"
            elif "excel" in mime_type or "spreadsheet" in mime_type or "sheet" in mime_type:
                file_ext = ".xlsx"

    # 3. DRM 제거 단계 (옵션)
    if remove_drm:
        try:
            drm_response = requests.post(
                "http://rpa-wlm-service-basic-prd.api.hcp01.skyhinix.com/rpa-wlm-service/v1/api/automations/schedule/chatbot",
                headers={"X-enviroment": "PRD"},
                json={
                    "CrUri": local_path,
                    "AutomationName": "DRMRemovalAutomation",
                    "FileName": os.path.basename(local_path),
                    "ScheduleType": "IMMEDIATE",
                    "BotType": "RPA",
                    "PoolName": "DefaultPool",
                    "UserInput": {}
                }
            )
            if drm_response.status_code != 200:
                print(f"DRM 제거 API 호출 실패: {drm_response.status_code}, {drm_response.text}")
            else:
                # API가 동기적으로 DRM 제거를 완료했다고 가정하고 진행
                pass  # 필요시 여기서 파일 처리 완료까지 대기 또는 확인 로직 추가
        except Exception as e:
            print("DRM 제거 API 호출 중 예외 발생:", e)

    # 4. 파일 유형별 이미지 추출
    os.makedirs(output_dir, exist_ok=True)
    if file_ext == ".pdf":
        images = extract_images_from_pdf(local_path, output_dir)
    elif file_ext == ".docx":
        images = extract_images_from_docx(local_path, output_dir)
    elif file_ext == ".pptx":
        images = extract_images_from_pptx(local_path, output_dir)
    elif file_ext == ".xlsx":
        images = extract_images_from_xlsx(local_path, output_dir)
    else:
        raise ValueError(f"지원되지 않는 파일 형식입니다: {file_ext}")
    return images
